In [ ]:
!pip install jschon

In [ ]:
!pip install js2py

In [1]:
from jschon import create_catalog,JSON, JSONSchema
import json,os
import pathlib
import inspect
import  js2py


In [2]:
create_catalog('2019-09')


Catalog('catalog')

In [3]:
keywords_to_avoid = ['not','anyOf','oneOf','allOf','patternPropreties']
def extract_errors_map(error:dict, error_map:dict):
    # print(error)
    # if there's errors in subschema treat them since they are the origin of the error  
    if 'errors' in error:
        for e in error['errors']:
            extract_errors_map(e, error_map)
    else : 
        keywordLocation = str(error['keywordLocation'])
        key = keywordLocation.split('/')[-1]
        
        if error["valid"] == True :
            if keywordLocation.__contains__("not"):
                if error_map.get(key) is None : 
                    error_map[key]=[error['error'] if 'error' in error else '']
                else : 
                    error_map[key].append(error['error'] if 'error' in error else '') 
                
            pass 
        else :      
            if key and  'error' in error and key not in keywords_to_avoid:
                if error_map.get(key) is None : 
                    error_map[key]=[error['error'] if 'error' in error else '']
                else : 
                    error_map[key].append(error['error'] if 'error' in error else '')
    
    # print("there's still error")
   

def extract_errors(errors): 
    error_map= {}
    for error in errors : 
        extract_errors_map(error,error_map)
    return list(error_map.keys())

In [24]:
import json
import subprocess
def migrate_schema(schema_migration_path,schema_path):
    try:
        # Convertir le schéma JSON en chaîne JSON
        schema_migration_path_str = json.dumps(schema_migration_path)
        schema_path_str = json.dumps(schema_path)
        # Appeler le script TypeScript avec le schéma JSON en tant qu'argument
        command = ['ts-node', '-e', f'require("./scripts/migrate").migrate({schema_migration_path_str},{schema_path_str})']
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE,shell=True, text=True)
        stdout, stderr = process.communicate()
        # Vérifier si une erreur s'est produite pendant l'exécution du script TypeScript
        if stderr:
            raise RuntimeError(f"Erreur lors de l'exécution du script TypeScript : {stderr}")
        return True

    except Exception as e:
        # print(str(e))
        print(f"Une erreur s'est produite : {e}")
        return None
def migrate(schemas_dir:str,schema_name:str): 
    schema_migration_path = f'{schemas_dir}/{schema_name}-migrated.json'
    schema_path = f'{schemas_dir}/{schema_name}.json'
    try : 
        schema_migrated = migrate_schema(schema_migration_path,schema_path)
        if schema_migrated is None :
            return False 
        return True
    except Exception as e: 
        print(e)
        return False

In [28]:
from tqdm import tqdm
def process_sub_dir(files_list) : 
    for subdir in tqdm(files_list) :
        schema_name = 'schema'
        schema_dir =f'{subdir}'
        print(f"Migrating : {schema_name}") 
        res = migrate(schema_dir,schema_name)
        if res : 
            print("Migrating correctly")
        else : 
            print("Could not migrate")

def migrate_schemas(): 
    files = [f'./dataset/{x}' for x in os.listdir("./dataset")[:-2]]
    process_sub_dir(files)
            

In [29]:
migrate_schemas()

  0%|          | 0/9 [00:00<?, ?it/s]

Migrating : schema


 11%|█         | 1/9 [00:02<00:22,  2.82s/it]

Migrating correctly
Migrating : schema


 22%|██▏       | 2/9 [00:05<00:18,  2.71s/it]

...

Migrating correctly
Migrating : schema


 89%|████████▉ | 8/9 [00:21<00:02,  2.62s/it]

Migrating correctly
Migrating : schema


100%|██████████| 9/9 [00:23<00:00,  2.62s/it]

Migrating correctly


In [30]:

def validate_instance(schema_dir:str,instance_dir): 
    with open(schema_dir,'r',encoding='utf-8') as f : 
        schema= json.load(f)
    print(schema)
    demo_schema = JSONSchema(schema)
    # print(demo_schema)
    with open (instance_dir,'r',encoding='utf-8') as f : 
        instance=json.load(f)
        
    result = demo_schema.evaluate(
        JSON(instance)
    )
    return result 
def get_errors_validation(schema_dir:str,instance_dir:str): 
    result = validate_instance(schema_dir,instance_dir)
    # print(result.output('detailed'))# if errors in result.output('detailed')
    if(result.output("verbose")['valid']==False or "errors" in result.output("verbose")): 
        errors = extract_errors(result.output('verbose')['errors'])
    else : 
        errors=[]
    return errors

In [32]:
src_file_path = os.path.dirname(os.path.abspath("__file__"))
schema_path = src_file_path+'/dataset/8/schema-migrated.json'
instance_path = src_file_path+'/dataset/8/instance.json'
# output = validate_instance(schema_path,instance_path).output('verbose')
# with open(src_file_path+'/dataset/8/errors.json','w') as f: 
#     json.dump(output,f)
validate_instance(schema_path,instance_path)

{'$schema': 'https://json-schema.org/draft/2019-09/schema', '$id': 'https://raw.githubusercontent.com/AlekseyBuzmakov/FCAPS/master/FCAPS/schemas/graph.json', 'title': 'Directed Graph JSON', 'description': 'Json file gives a description of a directed graph', 'type': 'array', 'prefixItems': [{'type': 'object', 'title': 'General Grap Info', 'description': 'An object contains general info about the graph', 'properties': {'NodesCount': {'title': 'Number of Nodes', 'description': 'The total number of nodes in the graph', 'type': 'integer', 'minimum': 0}, 'ArcsCount': {'title': 'Number of Arcs', 'description': 'The total number of arcs in the graph', 'type': 'integer', 'minimum': 0}}}, {'title': 'Nodes Info', 'description': 'All nodes are enumerated in this object', 'type': 'object', 'properties': {'Nodes': {'title': 'List of Nodes', 'description': 'The nodes of the graph are given within this array', 'type': 'array', 'items': {}}}, 'required': ['Nodes']}, {'title': 'Arcs Info', 'description'

In [34]:
src_file_path = os.path.dirname(os.path.abspath("__file__"))

# print(src_file_path)
dataset_dir =  src_file_path+'/dataset'
for dir in os.listdir(dataset_dir) : 
    if dir=='instances' or dir=='schemas' : 
        continue 
    print(f"Treating schema : {dir}")
    schema_dir = f'{dataset_dir}/{dir}/schema-migrated.json'
    instance_dir=f'{dataset_dir}/{dir}/instance.json'
    errors=get_errors_validation(schema_dir,instance_dir)
    print(errors)

Treating schema : 1
{'required': ['name'], 'properties': {'name': {'anyOf': [{'type': 'object', 'description': 'Cool! We got a string here!', 'required': ['a'], 'properties': {'a': {'type': 'object', 'properties': {'b': {'type': 'string'}}, 'required': ['b']}}}, {'type': 'integer'}]}}, '$schema': 'https://json-schema.org/draft/2019-09/schema'}
['required', 'type']
Treating schema : 2
{'required': ['something'], 'properties': {'something': {'required': ['name'], 'properties': {'name': {'anyOf': [{'type': 'object', 'description': 'Cool! We got a string here!', 'required': ['a'], 'properties': {'a': {'type': 'object', 'properties': {'b': {'type': 'string'}}, 'required': ['b'], 'minProperties': 2}}}, {'type': 'integer'}]}}}}, '$schema': 'https://json-schema.org/draft/2019-09/schema'}
['type', 'minProperties']
Treating schema : 3
{'type': 'object', 'properties': {'name': {'type': 'string'}}, 'not': {'required': ['name']}, '$schema': 'https://json-schema.org/draft/2019-09/schema'}
['type', '